In [23]:
import pandas as pd
import re
import io
import pdfplumber
import requests

In [28]:
ATTACHMENT_TYPE_ERR_MSG = (
    "All attachments must be either DctmObjRef or Attachment type, got {}: {}"
)
ATTACH_TYPE_EXPECTED = "Attachment expected to be of type `Attachment`, got {}"
UNEXPECTED_ATTR_TO_PARSE = (
    "Attribute to parse from attachments expected to be in "
    "['body', 'filename'], got '{}'"
)
DCTM_OBJ_REF_EXPECTED = "Expected DctmObjRef, got {}: {}"
DOXC2TXT_EXCEPTION = "Cannot process file, raised '{}' error"
LIST_OR_STR_ATTACH_EXPECTED = "Got type {} for attachment, only list or str accepted"
PAGE_SEP = "\n" + "=" * 31 + " NEW PAGE " + "=" * 31 + "\n"
MISSING_SPACES_PATTERNS = [
    "IndicativeTermsheet\n",
    "PRIVATEPLACEMENT\n",
    "PublicOfferingonlyin:",
]


def check_txt_missing_spaces(all_pages_txt: str, threshold: float = 0.06) -> bool:
    """Check if the parsed PDF has missing spaces (as for all Leonteq termsheets).

    Notes
    -----
    The alignment used to format the Leonteq termsheets are not properly recognized by
    our PDF converter. As an undesirable result, most spaces are being removed during
    the conversion step leading to erroneous extractions.
    """

    nb_spaces = all_pages_txt.count(" ")
    nb_chars = len(all_pages_txt)
    ratio = nb_spaces / nb_chars

    return ratio < threshold and any(
        p in all_pages_txt for p in MISSING_SPACES_PATTERNS
    )


def pdf_text_from_bytes(
    pdf_bytes_string: bytes,
    max_pages: int = 999,
    pages_sep: str = PAGE_SEP,
) -> str:
    """Convert the PDF byte representation to text."""
    try:
        # Pdfplumber returns empty string for UTF-8 encoded strings
        # (without any exception raised), only Latin-1 works
        # On the other hand, FastAPI requires UTF-8 strings in payloads,
        # so we assume UTF-8 string arrives here
        pdf_bytes_string = pdf_bytes_string.decode("UTF-8").encode("Latin1")
    except UnicodeDecodeError:
        # If the above command fails, we will assume the byte string
        # is already Latin1 encoded
        pass

    all_pages_txt = ""
    pages_list = []
    with pdfplumber.open(io.BytesIO(pdf_bytes_string)) as pdf:
        for page_idx in range(
            0, min(len(pdf.pages), max_pages)
        ):  # pylint: disable=invalid-name
            pages_list.append(pdf.pages[page_idx].extract_text() + pages_sep)
            all_pages_txt += pdf.pages[page_idx].extract_text() + pages_sep
    return pages_list


In [29]:

with open(
    "SEK_Green_Bond_Framework.pdf",
    "rb",
) as fobj:
    pdf_bytes_utf8 = fobj.read().decode("Latin1").encode("UTF-8")
    pdf_text = pdf_text_from_bytes(pdf_bytes_utf8)



def clean_text(text):

    text = text.lower()
    text = text.replace("\n", " ")
    text = text.replace("  ", " ")
    return text


text = clean_text(pdf_text)


AttributeError: 'list' object has no attribute 'lower'

In [31]:
len(pdf_text)

3

In [26]:
text

'green bond framework 1. eligible projects "eligible projects" means a selected pool of projects funded, in whole or in part, by sek that promote the transition to low-carbon and climate resilient growth as determined by sek in its sole and absolute discretion. eligible projects may include: - projects that target a reduction in emissions of greenhouse gases (ghg) into the atmosphere or absorption of them from the atmosphere (“mitigation projects”). - projects that target the preservation or advancement of adaptive capacity and resilience in order to reduce the vulnerability of human and natural systems due to the impacts of climate changes (“adaptation projects”). 1.1 mitigation projects a type of activity can be labeled as contributing to climate change mitigation if it is one among a set of options available to decision-makers and if it leads (according to the available information) to a relatively significant emission reduction as compared to the other options. an eligible mitigati

In [17]:
def read_keys(filename):
    mykey = None
    myorg = None
    
    try:
        with open(filename, 'r') as file:
            for line in file:
                if 'mykey' in line:
                    mykey = line.split('=')[1].strip().strip('"')
                elif 'myorg' in line:
                    myorg = line.split('=')[1].strip().strip('"')
    except FileNotFoundError:
        print(f"The file {filename} does not exist.")
    
    return mykey, myorg

# Reading values from keys.txt
filename = "keys"
mykey, myorg = read_keys(filename)


In [8]:
pretext = """the following is a document that illustrates an ESG framework under which an issuer issues financial instruments
---- document beginning ----
"""

In [12]:
posttext = """
---- document ending ----

I need you to answer these questions about the above text : 

---- questions about the document ----

1. The framework can mention the projects for which the funding will be used. What are those projects?
response structure: List of Eligible Project : None or the list if found

2. The framework can also mention the project categories for which the funding will be used. What are those categories?
response structure: List of Eligible Project Categories : None or the list if found

3. The framework can mention a selection process for the eligible projects. Does it mention it or not?
response structure : ESG project selection process: Y/N

4. the framework can mention a management of the proceeds in correspondance with the ESG initiative. Does it mention it or not?
This can be mentioned in the text under the format : "Allocation of proceeds"
response structure : Management of Proceeds : Y/N

"""

In [13]:
prompt = pretext + text + posttext

In [21]:
prompt = "hi how are you"

In [ ]:
mykey = ""

In [ ]:
import requests

url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {mykey}",
    "Content-Type": "application/json"
}
data = {
    "model": "gpt-3.5-turbo",
    "messages": [
        {"role": "user", "content": prompt},
    ]
}

try:
    response = requests.post(url, headers=headers, json=data, verify=False)
    response.raise_for_status()  # Raises an error for 4xx/5xx responses
    print(response.json()["choices"][0]["message"]["content"])
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")


An error occurred: 401 Client Error: Unauthorized for url: https://azure-openai-dev-001.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview


C:\Users\XL892PG\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'azure-openai-dev-001.openai.azure.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
